In [1]:
import pandas as pd
import numpy as np

from xgboost import XGBRegressor

from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

/Users/we/.local/share/virtualenvs/hy-research-wW9tVLY_/lib/python3.8/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [2]:
#import Common modult
from util import preprocess as pre

In [3]:
pre_processed = pre.getDataframe()
df = pre_processed.getpreprocessed()

/Users/we/Workspaces/ideyedi/hy-research/data/220117.2144.csv
/Users/we/Workspaces/ideyedi/hy-research/data/220223.1237.csv
/Users/we/Workspaces/ideyedi/hy-research/data/220319.2214.csv
/Users/we/Workspaces/ideyedi/hy-research/data/220331.0836.csv
/Users/we/Workspaces/ideyedi/hy-research/data/220117.2204.csv


### Modeling stage
전에 사용했던 XGboosting model을 사용

df를 모델링의 중점이 되는 데이터프레임으로 할당

In [4]:
# y: index for data sizes
y = [idx for idx in range(0, df.shape[0])]
# Test-Set 분리
x_train, x_test, y_train, y_test = train_test_split(df, y, test_size=0.5, random_state=10)
x_train.shape, x_test.shape

((789, 139), (789, 139))

In [5]:
x = x_train.drop('label', axis=1)
x_vali = x_test.drop('label', axis=1)

y = x_train['label']
y_vali = x_test['label']

In [6]:
# K-fold로 10개의 generator 생성
kf = KFold(n_splits =  10, shuffle = True, random_state = 96)
# Make Model
xgb = XGBRegressor(random_state = 256)

In [10]:
rmse_list = []
xgb_pred = np.zeros((x_train.shape[0]))

# K(10)-fold validation Loop
for tr_idx, val_idx in kf.split(x, y) :
    tr_x, tr_y = x.iloc[tr_idx], y.iloc[tr_idx]
    val_x, val_y = x.iloc[val_idx], y.iloc[val_idx]

    xgb.fit(tr_x, tr_y)

    pred = np.expm1([0 if x < 0 else x for x in xgb.predict(val_x)])
    sub_pred = np.expm1([0 if x < 0 else x for x in xgb.predict(x_vali)])
    rmse = np.sqrt(mean_squared_error(val_y, pred))


    rmse_list.append(rmse)
    xgb_pred += (sub_pred / 10)

/Users/we/.local/share/virtualenvs/hy-research-wW9tVLY_/lib/python3.8/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
/Users/we/.local/share/virtualenvs/hy-research-wW9tVLY_/lib/python3.8/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
/Users/we/.local/share/virtualenvs/hy-research-wW9tVLY_/lib/python3.8/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
/Users/we/.local/share/virtuale

In [12]:
print(f'MSE: {np.mean(rmse_list) * 100}%')

MSE: 74.46836590663469%
